In [10]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Sample data
texts = [
    "I love this product",
    "This is the worst service ever",
    "Absolutely fantastic experience"
]
labels = [1, 0, 1]

# Fit TF-IDF + logistic regression
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
model = LogisticRegression().fit(X, labels)

# Pick one instance to explain
instance_text = "I love this product"
instance_vector = vectorizer.transform([instance_text]).toarray()[0]
feature_names = vectorizer.get_feature_names_out()
print(feature_names)
# Choose a background vector (e.g., all zeros or mean TF-IDF)
background_vector = np.zeros_like(instance_vector)




['absolutely' 'ever' 'experience' 'fantastic' 'is' 'love' 'product'
 'service' 'the' 'this' 'worst']


In [3]:
background_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [4]:


# Simulate a binary mask: e.g., keep only 'love' and 'product'
keep = ["love", "product"]
mask = np.array([1 if word in keep else 0 for word in feature_names])



In [5]:
mask

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [7]:
# Create a perturbed version: masked features replaced by background
perturbed_vector = np.where(mask == 1, instance_vector, background_vector)
perturbed_vector

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.62276601, 0.62276601, 0.        , 0.        , 0.        ,
       0.        ])

In [8]:


# Show side-by-side
import pandas as pd

df = pd.DataFrame({
    "feature": feature_names,
    "original_tfidf": instance_vector,
    "mask": mask,
    "perturbed_tfidf": perturbed_vector
})

df

feature  original_tfidf  mask  perturbed_tfidf
0   absolutely        0.000000     0         0.000000
1         ever        0.000000     0         0.000000
2   experience        0.000000     0         0.000000
3    fantastic        0.000000     0         0.000000
4           is        0.000000     0         0.000000
5         love        0.622766     1         0.622766
6      product        0.622766     1         0.622766
7      service        0.000000     0         0.000000
8          the        0.000000     0         0.000000
9         this        0.473630     0         0.000000
10       worst        0.000000     0         0.000000

In [11]:

df = df[(df["original_tfidf"] > 0) | (df["perturbed_tfidf"] > 0)]
print(df.sort_values("original_tfidf", ascending=False))

# Predict
original_pred = model.predict_proba([instance_vector])[0,1]
perturbed_pred = model.predict_proba([perturbed_vector])[0,1]
delta = original_pred - perturbed_pred

print(f"\nOriginal Prediction: {original_pred:.4f}")
print(f"Perturbed Prediction (only 'love' and 'product'): {perturbed_pred:.4f}")
print(f"Change in prediction (contribution of masked features): {delta:.4f}")

   feature  original_tfidf  mask  perturbed_tfidf
5     love        0.622766     1         0.622766
6  product        0.622766     1         0.622766
9     this        0.473630     0         0.000000

Original Prediction: 0.7161
Perturbed Prediction (only 'love' and 'product'): 0.7204
Change in prediction (contribution of masked features): -0.0042


In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

print(es.info())

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.010s]


{'name': 'a11a2100a8aa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gAf831kRS1GKH2EiTlG_ug', 'version': {'number': '8.13.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '16cc90cd2d08a3147ce02b07e50894bc060a4cbf', 'build_date': '2024-04-05T14:45:26.420424304Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [10]:
%run ES_class.py

In [11]:

# Example usage and comprehensive testing
# if __name__ == "__main__":
try:
    # Initialize the hybrid IR system with all components enabled
    # Note: Make sure Elasticsearch is running on localhost:9200
    hybrid_ir = HybridInformationRetrieval(
        es_host="http://localhost:9200",  # Full URL with scheme
        enable_sparse=True,
        enable_dense=True,
        enable_reranker=True,
        lazy_load_models=True
    )
except ConnectionError as e:
    print(f"Error connecting to Elasticsearch: {e}")
    print("Please ensure Elasticsearch is running on http://localhost:9200")
    print("You can start Elasticsearch using Docker:")
    print("docker run -d --name elasticsearch -p 9200:9200 -e 'discovery.type=single-node' -e 'xpack.security.enabled=false' elasticsearch:8.11.0")
    exit(1)
except Exception as e:
    print(f"Unexpected error: {e}")
    exit(1)

# Sample documents for testing
sample_documents = [
    {
        "id": "doc1",
        "content": "Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.",
        "metadata": {"category": "AI", "author": "John Doe"}
    },
    {
        "id": "doc2", 
        "content": "Deep learning neural networks have revolutionized computer vision and natural language processing.",
        "metadata": {"category": "Deep Learning", "author": "Jane Smith"}
    },
    {
        "id": "doc3",
        "content": "Information retrieval systems help users find relevant documents from large collections.",
        "metadata": {"category": "IR", "author": "Bob Johnson"}
    },
    {
        "id": "doc4",
        "content": "Elasticsearch is a distributed search and analytics engine built on Apache Lucene.",
        "metadata": {"category": "Search", "author": "Alice Wilson"}
    },
    {
        "id": "doc5",
        "content": "Sentence transformers create dense vector representations of text for semantic search.",
        "metadata": {"category": "NLP", "author": "Charlie Brown"}
    }
]


INFO:elastic_transport.transport:HEAD http://localhost:9200/ [status:200 duration:0.185s]
INFO:__main__:Successfully connected to Elasticsearch at http://localhost:9200
INFO:elastic_transport.transport:HEAD http://localhost:9200/hybrid_search_index [status:404 duration:0.002s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index [status:200 duration:0.145s]
INFO:__main__:Created index: hybrid_search_index


In [16]:

# Add documents to the index
hybrid_ir.add_documents(sample_documents)

query = "machine learning algorithms for search"

# Test 1: Full pipeline with intermediate outputs
print("="*80)
print("TEST 1: FULL HYBRID SEARCH WITH INTERMEDIATE OUTPUTS")
print("="*80)

full_output = hybrid_ir.hybrid_search(
    query, 
    sparse_top_k=10, 
    dense_top_k=10, 
    final_top_k=3, 
    return_intermediate=False
)
hybrid_ir.display_hybrid_output(full_output, query)

# Test 2: Disable dense search and rerun
print("\n" + "="*80)
print("TEST 2: SPARSE SEARCH ONLY (DENSE DISABLED)")
print("="*80)

hybrid_ir.disable_component(SearchComponent.DENSE)
sparse_only_output = hybrid_ir.hybrid_search(query, return_intermediate=True)
hybrid_ir.display_hybrid_output(sparse_only_output, query)


INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc1 [status:200 duration:0.022s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc2 [status:200 duration:0.006s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc3 [status:200 duration:0.003s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc4 [status:200 duration:0.005s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc5 [status:200 duration:0.005s]
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_refresh [status:200 duration:0.006s]
INFO:__main__:Added 5 documents to index
INFO:__main__:Starting hybrid search for query: 'machine learning algorithms for search'
INFO:__main__:Enabled components: ['sparse', 'reranker']
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_search [status:200 durat

TEST 1: FULL HYBRID SEARCH WITH INTERMEDIATE OUTPUTS


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s]
INFO:__main__:Reranked and returned top 3 results
INFO:__main__:Disabled component: dense
INFO:__main__:Starting hybrid search for query: 'machine learning algorithms for search'
INFO:__main__:Enabled components: ['sparse', 'reranker']
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_search [status:200 duration:0.011s]
INFO:__main__:Sparse search returned 4 results
INFO:__main__:Dense search skipped (disabled)
INFO:__main__:After deduplication: 4 unique results



COMPLETE HYBRID SEARCH PIPELINE FOR: 'machine learning algorithms for search'

RESULT SUMMARY:
  sparse_count: 4
  dense_count: 0
  deduplicated_count: 4
  final_count: 3

SPARSE SEARCH RESULTS (BM25)

1. Document ID: doc1
   Score: 3.3349 (sparse)
   Content: Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.
   --------------------------------------------------

2. Document ID: doc5
   Score: 2.3566 (sparse)
   Content: Sentence transformers create dense vector representations of text for semantic search.
   --------------------------------------------------

3. Document ID: doc2
   Score: 0.8814 (sparse)
   Content: Deep learning neural networks have revolutionized computer vision and natural language processing.
   --------------------------------------------------

4. Document ID: doc4
   Score: 0.8814 (sparse)
   Content: Elasticsearch is a distributed search and analytics engine built on Apache Lucene.
   -----------------

Batches: 100%|██████████| 1/1 [00:00<00:00, 105.37it/s]
INFO:__main__:Reranked and returned top 3 results



COMPLETE HYBRID SEARCH PIPELINE FOR: 'machine learning algorithms for search'

RESULT SUMMARY:
  sparse_count: 4
  dense_count: 0
  combined_count: 4
  deduplicated_count: 4
  final_count: 3

SPARSE SEARCH RESULTS (BM25)

1. Document ID: doc1
   Score: 3.3349 (sparse)
   Content: Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.
   --------------------------------------------------

2. Document ID: doc5
   Score: 2.3566 (sparse)
   Content: Sentence transformers create dense vector representations of text for semantic search.
   --------------------------------------------------

3. Document ID: doc2
   Score: 0.8814 (sparse)
   Content: Deep learning neural networks have revolutionized computer vision and natural language processing.
   --------------------------------------------------

4. Document ID: doc4
   Score: 0.8814 (sparse)
   Content: Elasticsearch is a distributed search and analytics engine built on Apache Lucene.


In [15]:
SearchComponent.DENSE

<SearchComponent.DENSE: 'dense'>

In [ ]:

# Test 3: Enable dense, disable reranker
print("\n" + "="*80)
print("TEST 3: SPARSE + DENSE ONLY (RERANKER DISABLED)")
print("="*80)

hybrid_ir.enable_component(SearchComponent.DENSE)
hybrid_ir.disable_component(SearchComponent.RERANKER)
no_rerank_output = hybrid_ir.hybrid_search(query, return_intermediate=True)
hybrid_ir.display_hybrid_output(no_rerank_output, query)

# Test 4: Single component searches
print("\n" + "="*80)
print("TEST 4: SINGLE COMPONENT SEARCHES")
print("="*80)

# Re-enable all components for single component tests
hybrid_ir.enable_component(SearchComponent.RERANKER)

sparse_results = hybrid_ir.search_with_single_component(query, SearchComponent.SPARSE, top_k=5)
hybrid_ir.display_results(sparse_results, title="SPARSE ONLY")

dense_results = hybrid_ir.search_with_single_component(query, SearchComponent.DENSE, top_k=5)
hybrid_ir.display_results(dense_results, title="DENSE ONLY")

# Test 5: Toggle components dynamically
print("\n" + "="*80)
print("TEST 5: DYNAMIC COMPONENT TOGGLING")
print("="*80)

print("Current enabled components:", [comp.value for comp in hybrid_ir.get_enabled_components()])

hybrid_ir.toggle_component(SearchComponent.SPARSE)
print("After toggling sparse:", [comp.value for comp in hybrid_ir.get_enabled_components()])

hybrid_ir.toggle_component(SearchComponent.SPARSE)  # Toggle back
print("After toggling sparse again:", [comp.value for comp in hybrid_ir.get_enabled_components()])